# MySQL Connector Python을 이용한 MySQL 사용

## 의존성 설치

In [1]:
# mysql 8 이상
%pip install mysql-connector-python

# 하위 버전
# %pip install mysql.connector

Note: you may need to restart the kernel to use updated packages.


In [2]:
DB_SETTING = {
  "host": "localhost",
  "port": "3808",
  "user": "root",
  "password": "123qweasd.",
}

DB_NAME = "sql_python_test"

## DBMS와 연결하기

In [3]:
from mysql import connector

def get_mysql_connection(setting) -> connector.MySQLConnection:
  return connector.Connect(**setting)

In [4]:
with get_mysql_connection(DB_SETTING) as mysql_conn:
  cursor = mysql_conn.cursor()
  cursor.execute("SHOW DATABASES")

  for db in cursor:
    print (db)

('information_schema',)
('mysql',)
('performance_schema',)
('sql_orm_test',)
('sql_test',)
('sys',)


## DB 만들기

In [5]:
with get_mysql_connection(DB_SETTING) as mysql_conn:
  cursor = mysql_conn.cursor()
  cursor.execute("SHOW DATABASES")
  exist_flag = False
  for name_tuple in cursor:
    exist_flag = exist_flag or DB_NAME in name_tuple

  if not exist_flag:
    # 반드시 강력한 확인 후 사용해야하는 문장
    cursor.execute(f"CREATE DATABASE {DB_NAME}")

In [6]:
with get_mysql_connection(DB_SETTING) as mysql_conn:
  cursor = mysql_conn.cursor()
  cursor.execute("SHOW DATABASES")

  for db in cursor:
    print (db)

('information_schema',)
('mysql',)
('performance_schema',)
('sql_orm_test',)
('sql_python_test',)
('sql_test',)
('sys',)


## DB와 연결하기

In [7]:
DB_SETTING['database'] = DB_NAME

## 테이블 만들기

In [8]:
mysql_conn = get_mysql_connection(DB_SETTING)

In [9]:
cursor = mysql_conn.cursor(buffered=True)

USER_CREATE_SQL = \
"""
CREATE TABLE user (
  id int AUTO_INCREMENT,
  name VARCHAR(20) NOT NULL,
  age INT NOT NULL,
  birthday DATE,
  reg_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
  PRIMARY KEY (id)
);
"""

cursor.execute(USER_CREATE_SQL)

In [10]:
cursor.execute("SHOW TABLES")
for x in cursor:
  print (x)

('user',)


## 생 트랜잭션 날리기

In [11]:
mysql_conn.start_transaction()
print ("cmd_query insert result:", mysql_conn.cmd_query("INSERT INTO user (name, age, birthday) VALUES ('황씨', '26', '1996-02-21')"))
mysql_conn.commit()

cmd_query insert result: {'field_count': 0, 'affected_rows': 1, 'insert_id': 1, 'status_flag': 1, 'warning_count': 0}


In [12]:
mysql_conn.start_transaction()
print ("cmd_query insert result:", mysql_conn.cmd_query("INSERT INTO user (name, age, birthday) VALUES ('구씨', '26', '1996-02-21')"))
mysql_conn.rollback()

cmd_query insert result: {'field_count': 0, 'affected_rows': 1, 'insert_id': 2, 'status_flag': 1, 'warning_count': 0}


## 커서를 이용하여 트랜잭션 날리기

In [13]:
mysql_cursor = mysql_conn.cursor(buffered=True)
mysql_cursor.execute("SELECT * FROM user")
print (mysql_cursor.fetchall())
mysql_cursor.close()

[(1, '황씨', 26, datetime.date(1996, 2, 21), datetime.datetime(2022, 5, 31, 10, 4, 42))]


True

In [14]:
mysql_cursor = mysql_conn.cursor(buffered=True)
mysql_cursor.execute("SELECT * FROM user")
# fetchall
for row in mysql_cursor:
  print (row)
mysql_cursor.close()

(1, '황씨', 26, datetime.date(1996, 2, 21), datetime.datetime(2022, 5, 31, 10, 4, 42))


True

In [15]:
mysql_cursor = mysql_conn.cursor(buffered=True)
mysql_cursor.execute("SELECT * FROM user")
print (mysql_cursor.fetchone())
mysql_cursor.close()

(1, '황씨', 26, datetime.date(1996, 2, 21), datetime.datetime(2022, 5, 31, 10, 4, 42))


True

## 가짜 데이터를 이용한 테스팅 해보기

In [16]:
%pip install Faker

Note: you may need to restart the kernel to use updated packages.


In [17]:
from faker import Faker
import datetime
Faker.seed(42)

In [18]:
faker = Faker(locale='ko_KR')


In [19]:
def get_date_string_from_date(date: datetime.date):
  return "%04d-%02d-%02d" % (date.year, date.month, date.day)

def gen_fake_user_data():
  name = faker.name()
  birthday = faker.date_between(start_date=datetime.datetime.today() - datetime.timedelta(days=10000), end_date=datetime.datetime.today())
  age_dt = datetime.datetime.today() - datetime.datetime.combine(birthday, datetime.datetime.min.time()) 

  age = age_dt.days // 365
  return { "name" : name, "birthday": get_date_string_from_date(birthday), "age": age, }


In [20]:
gen_fake_user_data()

{'name': '김수민', 'birthday': '2002-08-17', 'age': 19}

In [21]:
def create_values_from_data(data):
  return f"('{data['name']}', '{data['age']}', '{data['birthday']}')"

def generate_sql_str_values(count=50):
  l = []
  for _ in range(count):
    data_str = create_values_from_data(gen_fake_user_data())
    l.append(data_str)

  return ', '.join(l)

In [22]:
generate_sql_str_values(5)

"('김정호', '8', '2013-08-03'), ('이경수', '19', '2002-06-22'), ('조영호', '3', '2019-05-25'), ('정서윤', '7', '2015-01-31'), ('문정훈', '21', '2000-06-19')"

In [23]:
mysql_cursor: connector.connection.CursorBase = mysql_conn.cursor(buffered=True)

In [24]:
mysql_cursor.execute("INSERT INTO user (name, age, birthday) VALUES %s" % generate_sql_str_values())
# 이거 안하면 확정이 안됨
mysql_conn.commit()

In [25]:
mysql_cursor.execute("SELECT * FROM user")
# fetchall
for row in mysql_cursor:
  print (row)

(1, '황씨', 26, datetime.date(1996, 2, 21), datetime.datetime(2022, 5, 31, 10, 4, 42))
(3, '이민지', 1, datetime.date(2021, 1, 4), datetime.datetime(2022, 5, 31, 10, 4, 44))
(4, '김동현', 15, datetime.date(2006, 9, 26), datetime.datetime(2022, 5, 31, 10, 4, 44))
(5, '송지민', 9, datetime.date(2013, 4, 12), datetime.datetime(2022, 5, 31, 10, 4, 44))
(6, '유도현', 17, datetime.date(2005, 1, 3), datetime.datetime(2022, 5, 31, 10, 4, 44))
(7, '강채원', 7, datetime.date(2014, 9, 5), datetime.datetime(2022, 5, 31, 10, 4, 44))
(8, '김정남', 1, datetime.date(2021, 5, 3), datetime.datetime(2022, 5, 31, 10, 4, 44))
(9, '김상호', 23, datetime.date(1998, 6, 21), datetime.datetime(2022, 5, 31, 10, 4, 44))
(10, '박준영', 21, datetime.date(2000, 7, 26), datetime.datetime(2022, 5, 31, 10, 4, 44))
(11, '김상훈', 4, datetime.date(2018, 4, 16), datetime.datetime(2022, 5, 31, 10, 4, 44))
(12, '강보람', 2, datetime.date(2019, 11, 4), datetime.datetime(2022, 5, 31, 10, 4, 44))
(13, '박서준', 5, datetime.date(2016, 10, 22), datetime.datetime(

In [26]:
def execute_query_with_print(query, params=()):
  mysql_cursor: connector.connection.CursorBase = mysql_conn.cursor(buffered=True)
  mysql_cursor.execute(query, params=params)
  for x in mysql_cursor:
    print (x)
  mysql_cursor.close()



In [27]:

execute_query_with_print("""
SELECT *
FROM user
WHERE age < 30
ORDER BY name DESC, age ASC
""")

(1, '황씨', 26, datetime.date(1996, 2, 21), datetime.datetime(2022, 5, 31, 10, 4, 42))
(31, '허은서', 10, datetime.date(2011, 8, 29), datetime.datetime(2022, 5, 31, 10, 4, 44))
(24, '최진우', 4, datetime.date(2018, 3, 7), datetime.datetime(2022, 5, 31, 10, 4, 44))
(44, '최예은', 0, datetime.date(2021, 9, 23), datetime.datetime(2022, 5, 31, 10, 4, 44))
(20, '최성호', 22, datetime.date(1999, 9, 28), datetime.datetime(2022, 5, 31, 10, 4, 44))
(19, '최상훈', 7, datetime.date(2014, 12, 3), datetime.datetime(2022, 5, 31, 10, 4, 44))
(40, '최상현', 19, datetime.date(2002, 7, 11), datetime.datetime(2022, 5, 31, 10, 4, 44))
(38, '지재호', 12, datetime.date(2009, 12, 9), datetime.datetime(2022, 5, 31, 10, 4, 44))
(52, '전현주', 1, datetime.date(2020, 9, 2), datetime.datetime(2022, 5, 31, 10, 4, 44))
(25, '장옥자', 18, datetime.date(2004, 2, 10), datetime.datetime(2022, 5, 31, 10, 4, 44))
(23, '이현준', 9, datetime.date(2013, 1, 13), datetime.datetime(2022, 5, 31, 10, 4, 44))
(48, '이준호', 18, datetime.date(2003, 6, 30), datetime

In [28]:

execute_query_with_print("""
SELECT name, COUNT(id) as id_count
FROM user
GROUP BY name
ORDER BY id_count DESC
""")

('김지훈', 2)
('김상호', 1)
('이은지', 1)
('장옥자', 1)
('김영호', 1)
('오도윤', 1)
('박민수', 1)
('김동현', 1)
('강보람', 1)
('최성호', 1)
('권예진', 1)
('박예준', 1)
('최예은', 1)
('강채원', 1)
('김경희', 1)
('이현준', 1)
('허은서', 1)
('박선영', 1)
('윤재호', 1)
('황씨', 1)
('박준영', 1)
('윤민서', 1)
('이수진', 1)
('박도윤', 1)
('고명자', 1)
('고미숙', 1)
('송지민', 1)
('박서준', 1)
('김명숙', 1)
('이은주', 1)
('권정순', 1)
('이동현', 1)
('김정남', 1)
('최진우', 1)
('남중수', 1)
('최상현', 1)
('이준호', 1)
('이민지', 1)
('김상훈', 1)
('최상훈', 1)
('곽중수', 1)
('김은지', 1)
('박영철', 1)
('전현주', 1)
('유도현', 1)
('김순옥', 1)
('김정수', 1)
('이민준', 1)
('지재호', 1)
('이영수', 1)


In [29]:

execute_query_with_print("""
SELECT name, COUNT(id) as id_count
FROM user
GROUP BY name
HAVING id_count > 1
ORDER BY id_count DESC
""")

('김지훈', 2)
